In [180]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [181]:
from IPython.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
#InteractiveShell.ast_node_interactivity = "last"
from importlib import reload
import pandas as pd
%autoreload 2
pd.set_option("display.precision",2)

In [182]:
import pandas as pd
df = pd.read_pickle("irradiance2014.pickle")['2013']
reload(bau)


<module 'bau' from '/home/at/Dropbox/Exergy/Project/HouseModels/bau.py'>

In [187]:
import bau
import hsolar
import hp
import tank
import boiler

# Every combination starts with BAU and ends with boiler.
ba = bau.BAU()
bo = boiler.Boiler()

# Here are the other possible devices
devs = [hsolar.HSolar(),
        # tank.Tank(),  Very slow!
        hp.HP()
       ]

devnames = []
for d in devs:
    devnames.append(type(d).__name__)

mat = pd.DataFrame(columns=devnames)
mat.columns = devnames
mat['Cons']=0

In [189]:
from itertools import combinations
namecombos = []
devcombos = []
for i in range(0,len(devs)+1):
    namecombos.extend( combinations(devnames, i)) 
    devcombos.extend( combinations(devs, i)) 
    
devcombos    

[(),
 (<hsolar.HSolar at 0x7fbba79bab70>,),
 (<tank.Tank at 0x7fbba7a57320>,),
 (<hp.HP at 0x7fbba7a57438>,),
 (<hsolar.HSolar at 0x7fbba79bab70>, <tank.Tank at 0x7fbba7a57320>),
 (<hsolar.HSolar at 0x7fbba79bab70>, <hp.HP at 0x7fbba7a57438>),
 (<tank.Tank at 0x7fbba7a57320>, <hp.HP at 0x7fbba7a57438>),
 (<hsolar.HSolar at 0x7fbba79bab70>,
  <hp.HP at 0x7fbba7a57438>)]

In [190]:
mat

,HSolar,Tank,HP,Cons


In [186]:
i=0
for dl in devcombos:
    # Run BAU
    ba.apply(df)

    # Populate the "which devices ran" cells
    mat.loc[i,:]=0
    for d in dl:
        n = type(d).__name__
        mat.loc[i,n]=1
    
        # Actually run those devices
        d.apply(df)
    
    # Run the Boiler, set consumption
    bo.apply(df)
    mat.loc[i,'Cons'] = df.cons.sum()
    mat.loc[i,'Sav'] = df.bcons.sum() - df.cons.sum()
    i=i+1
        
mat        
    



,HSolar,Tank,HP,Cons,Sav
0,0.0,0.0,0.0,15586.85,0.00
1,1.0,0.0,0.0,14685.80,901.05
2,0.0,1.0,0.0,15586.85,0.00
3,0.0,0.0,1.0,11417.97,4168.88
4,1.0,1.0,0.0,14670.81,916.04
5,1.0,0.0,1.0,11188.77,4398.08
6,0.0,1.0,1.0,11417.97,4168.88
7,1.0,1.0,1.0,11185.47,4401.39


In [192]:
mat

,HSolar,Tank,HP,Cons
